In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# LinearRunOnce

The simplest linear solver in OpenMDAO is LinearRunOnce, which sequentially calls `apply_linear`
and `solve_linear` once on each subsystem. It is directly analogous to applying a single pass of the
chain rule to the whole system without any iteration at the top level. This linear solver can only be
used in systems where the following conditions are satisfied:

1. System does not contain a cycle, though subsystems may.
2. System does not contain any implicit states, though subsystems may.

However, subsystems can contain cycles or implicit states, provided that they are using the appropriate
solver such as [ScipyKrylov](../../../_srcdocs/packages/solvers.linear/scipy_iter_solver).

Here is an example of using LinearRunOnce to calculate the derivatives across a simple model with
the `Paraboloid` component.

In [2]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src28", get_code("openmdao.test_suite.components.paraboloid.Paraboloid"), display=False)

:::{Admonition} `Paraboloid` class definition 
:class: dropdown

{glue:}`code_src28`
:::

In [3]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid(), promotes=['x', 'y', 'f_xy'])

model.linear_solver = om.LinearRunOnce()

prob.setup(check=False, mode='fwd')

prob.set_val('x', 0.0)
prob.set_val('y', 0.0)

prob.run_model()

of = ['f_xy']
wrt = ['x', 'y']
derivs = prob.compute_totals(of=of, wrt=wrt, return_format='dict')

print(derivs['f_xy']['x'], [[-6.0]])
print(derivs['f_xy']['y'], [[8.0]])

[[-6.]] [[-6.0]]
[[8.]] [[8.0]]


In [4]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(derivs['f_xy']['x'], [[-6.0]], 1e-6)
assert_near_equal(derivs['f_xy']['y'], [[8.0]], 1e-6)

0.0

## LinearRunOnce Options

In [5]:
om.show_options_table("openmdao.solvers.linear.linear_runonce.LinearRunOnce")

Option,Default,Acceptable Values,Acceptable Types,Description
aitken_initial_factor,1.0,N/A,N/A,initial value for Aitken relaxation factor
aitken_max_factor,1.5,N/A,N/A,upper limit for Aitken relaxation factor
aitken_min_factor,0.1,N/A,N/A,lower limit for Aitken relaxation factor
assemble_jac,False,"[True, False]",['bool'],Activates use of assembled jacobian by this solver.
iprint,1,N/A,['int'],whether to print output
use_aitken,False,"[True, False]",['bool'],set to True to use Aitken relaxation


## LinearRunOnce Constructor

The call signature for the `LinearRunOnce` constructor is:

```{eval-rst}
    .. automethod:: openmdao.solvers.linear.linear_runonce.LinearRunOnce.__init__
        :noindex:
```